In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras_self_attention import SeqSelfAttention
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



Using TensorFlow backend.


In [4]:
train_Tb = Table()


dev_Tb = Table()
test_Tb=Table()

list1 = []
list2 = []




with open("dataset/PAWS_Wiki/train.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        train_Tb=train_Tb.with_columns(list1[0][ele], list2[ele])


train_Tb.show(5)
list1 = []
list2 = []

with open("dataset/PAWS_Wiki/dev.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        dev_Tb=dev_Tb.with_columns(list1[0][ele], list2[ele])


dev_Tb.show(5)

list1 = []
list2 = []

with open("dataset/PAWS_Wiki/test.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        test_Tb=test_Tb.with_columns(list1[0][ele], list2[ele])
        
test_Tb.show(5)

id,sentence1,sentence2,label
1,"In Paris , in October 1560 , he secretly met the English ...","In October 1560 , he secretly met with the English ambas ...",0
2,The NBA season of 1975 -- 76 was the 30th season of the ...,The 1975 -- 76 season of the National Basketball Associa ...,1
3,"There are also specific discussions , public profile deb ...","There are also public discussions , profile specific dis ...",0
4,"When comparable rates of flow can be maintained , the re ...",The results are high when comparable flow rates can be m ...,1
5,It is the seat of Zerendi District in Akmola Region .,It is the seat of the district of Zerendi in Akmola region .,1


id,sentence1,sentence2,label
1,Bradd Crellin represented BARLA Cumbria on a tour of Aus ...,Bradd Crellin also represented BARLA Great Britain on a ...,0
2,They were there to enjoy us and they were there to pray ...,They were there for us to enjoy and they were there for ...,1
3,"After the end of the war in June 1902 , Higgins left Sou ...","In August , after the end of the war in June 1902 , Higg ...",1
4,From the merger of the Four Rivers Council and the Audub ...,Shawnee Trails Council was formed from the merger of the ...,1
5,The group toured extensively and became famous in Israel ...,The group toured extensively and was famous in Israel an ...,1


id,sentence1,sentence2,label
1,"This was a series of nested angular standards , so that ...","This was a series of nested polar scales , so that measu ...",0
2,His father emigrated to Missouri in 1868 but returned wh ...,"His father emigrated to America in 1868 , but returned w ...",0
3,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",1
4,"Steiner argued that , in the right circumstances , the s ...",Steiner held that the spiritual world can be researched ...,0
5,"Luciano Williames Dias ( born July 25 , 1970 ) is a Braz ...",Luciano Williames Dias ( born 25 July 1970 ) is a former ...,0


In [5]:
ppdb_Tb=Table.read_table("dataset/ppdb/sanitized.csv")

In [6]:
train_id = train_Tb.column("id")
train_sen1=train_Tb.column("sentence1")
train_sen2=train_Tb.column("sentence2")  #PAWS_QQP
train_row = train_Tb.num_rows
dev_id=dev_Tb.column("id")
dev_row=dev_Tb.num_rows
dev_sen1=dev_Tb.column("sentence1")
dev_sen2=dev_Tb.column("sentence2")
test_id=test_Tb.column("id")
test_row=test_Tb.num_rows
test_sen1=test_Tb.column("sentence1")
test_sen2=test_Tb.column("sentence2")
train_label = train_Tb.column("label")
dev_label=dev_Tb.column("label")
test_label=test_Tb.column("label")

phrase = ppdb_Tb.column("PHRASE")
paraphrase=ppdb_Tb.column("PARAPHRASE")   #ppdb


In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(train_id)):   #index of PAWS_Wiki training set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in train_sen1[x] and paraphrase[z] in train_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, train_sen1[x])
            sent2=np.append(sent2, train_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, train_label[x])

 most valuable 
 's most 
 are generally 
 are generally 
 are generally 
 are generally 
 are generally 
 are generally 
 most popular 
 taken part in 
 even human 
 years later 
 highly significant 
 passed away 
 passed 
 two years 
 previously published 
 some time 
 world 's 
 logistics support 
 resolve the 
 resolve the 
 resolve the 
 secondary schools in 
 only possible 
 to verify 
 and historical 
 and historical 
 very hot 
 since then 
 two years 
 around the world 
 around the world 
 very long 
 very long 
 very nice 
 very nice 
 table of contents 
 , representing the 
 and supported 
 've created 
 later in life 
 well received 
 well received 
 received 
 years later 
 two years 
 second place 
 primary school 
 reached agreement 
 second place 
 nearly four 
 first time 
 very poor 
 two years 
 nuclear power plants 
 nuclear power 
 nuclear power 
 relationships are 
 the southern part 
 are typically 
 are typically 
 are typically 
 are typically 
 are typically 


In [10]:
phrase_dataset = Table().with_column("TRAIN_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [11]:
phrase_dataset.show(4)

TRAIN_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label
21,160456,"For their performances in the game , quarterback Jameis ...",Quarterback P. J. Williams and Defensive Back Jameis Win ...,most valuable,valuable,0
21,243017,"For their performances in the game , quarterback Jameis ...",Quarterback P. J. Williams and Defensive Back Jameis Win ...,'s most,most,0
33,395059,The leaves are generally 1.5-4 mm long and 0.2-0.7 mm wide .,The leaves are usually 1.5-4 mm long and 0.2-0.7 mm wide .,are generally,are usually,1
33,395061,The leaves are generally 1.5-4 mm long and 0.2-0.7 mm wide .,The leaves are usually 1.5-4 mm long and 0.2-0.7 mm wide .,are generally,are usually,1


In [ ]:
phrase_dataset.to_csv("dataset/Phrase_Level/PAWS_Wiki/PPDB_PAWS_Wiki_TRAIN.tsv", sep='\t')

In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(dev_id)):   #index of PAWS_Wiki dev set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in dev_sen1[x] and paraphrase[z] in dev_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, dev_sen1[x])
            sent2=np.append(sent2, dev_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, dev_label[x])

In [ ]:
phrase_dataset = Table().with_column("DEV_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [ ]:
phrase_dataset.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.csv", sep='\t')

In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(test_id)):   #index of PAWS_Wiki test set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in test_sen1[x] and paraphrase[z] in test_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, test_sen1[x])
            sent2=np.append(sent2, test_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, test_label[x])

In [ ]:
phrase_dataset = Table().with_column("TEST_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)
phrase_dataset.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.csv", sep='\t')

In [71]:
dataframe = pd.read_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TRAIN-Copy1.csv")

In [72]:
dataframe.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TRAIN.tsv", sep='\t', index=False)

In [73]:
dataframe = pd.read_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV-Copy1.csv")
dataframe.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.tsv", sep='\t', index=False)

In [56]:
list1=[]
list2=[]

label_data = Table()
with open("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.tsv", 'r') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(6):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        label_data=label_data.with_columns(list1[0][ele], list2[ele])



label_data.show(4)



DEV_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2
3.0,84905.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality
3.0,129607.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality
11.0,72492.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with
11.0,366448.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with


In [57]:

sent1=label_data.column("sent1")
sent2=label_data.column("sent2")
para1=label_data.column("paraphrase1")
para2=label_data.column("paraphrase2")

gg = [x.split() for x in sent1]
hh = [x.split() for x in sent2]
ii = [x.split() for x in para1]
jj = [x.split() for x in para2]



22

In [58]:
labels1=[]
for x in range(len(gg)):
    labels1.append(spit_label(gg[x], ii[x], len(ii[x]),len(ii[x]), []))



In [59]:
labels2=[]
for x in range(len(hh)):
    labels2.append(spit_label(hh[x], jj[x], len(jj[x]),len(jj[x]), []))



In [60]:
joined1=[]
for x in labels1:
    joined1.append(" ".join(x))

In [61]:
joined2=[]
for x in labels2:
    joined2.append(" ".join(x))

In [62]:
label_data

DEV_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2
3.0,84905.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality
3.0,129607.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality
11.0,72492.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with
11.0,366448.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with
22.0,543619.0,"b""I 'm currently working for a Big 4 as an Advanced Tax ...","b""I 'm currently working for a Big 4 as an Advanced Tax ...",much better,much
25.0,487239.0,"b'In Nevada when pulling out of a parking lot , can you ...","b'In Nevada when pulling out of a parking lot , can you ...",far left,left
29.0,284980.0,"b""What are the reasons for SAARC 's failure as regional ...","b""What are the reasons for SAARC 's failure as common or ...",economic cooperation,cooperation
56.0,459668.0,"b'I want to be a child psychologist , what qualification ...","b'I want to become a child psychologist , what qualifica ...",become one,be
57.0,71262.0,b'Why are beautiful girls so rude all the time ? I saw a ...,b'Why are rude girls so beautiful all the time ? I saw a ...,very rude,rude
57.0,494710.0,b'Why are beautiful girls so rude all the time ? I saw a ...,b'Why are rude girls so beautiful all the time ? I saw a ...,go away,away


In [63]:
label_data=label_data.with_columns("labels1", joined1, "labels2", joined2)
label_data=label_data.to_df()
label_data.to_csv("dataset/Phrase_Level/LABELED_PAWS_QQP_DEV.tsv", sep='\t', index=False)

In [34]:
new_Tb

In [15]:
    def spit_label(split_sent, para_split, ori_length, length, listy):
       
    
        if para_split[0]==split_sent[0]:
            
            if ori_length==1:
                listy.append("S")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy    
            
            elif ori_length==length:
                listy.append("B")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
            elif length==1:
                listy.append("E")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy
            
            else:
                listy.append("I")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
        else:
            listy.append("O")
            return spit_label(split_sent[1:], para_split, ori_length, length, listy)

In [88]:
bbb="".join("he ")

In [89]:
bbb

'he '